# ObservePoint API Management to Scale Governance for a large scale of Digital Assets 
## Create ObservePoint Audits based on Templat Audit created in ObservePoint UI

1. [Load Libraries](#1.-Load-Libraries-needed)
1. [Setup](#2.-Setup-API-&-Template-Audit-ID)
1. [API Call](#3.-API-Call)
1. [Check if Domain exists](#4.-Check-if-Domain-exists)
1. [Check if Audit exists](#5.-Check-if-Audit-exists)
1. [Create Domain](#6.-Create-Domain)
1. [Create Audit](#7.-Create-Audit)
1. [Get all Domains](#8.-Get-all-Domains-in-ObservePoint)
1. [Get all Audits](#9.-Get-all-Audits-in-ObservePoint)
1. [Create Audits](#10.-Create-Audits)

### 1. Load Libraries needed

In [ ]:
# If those modules are not installed in your environment 
# !pip install requests pandas

In [47]:
import requests as req
import pandas as pd
import json
import operator
import time
import random

### 2. Setup API & Template Audit ID

In [ ]:
apiKey = "API_KEY"
api = "https://api.observepoint.com/v2/"

template_audit_id = TEMPLATE_ID

### 3. API Call

In [34]:
def get_data(endpoint,method="GET",payload={}):
    headers = { 
                'authorization': "api_key " + apiKey,
                'accept': 'application/json'
    }
    
    if method in ["POST", "PUT"]:
        headers['Content-Type'] = 'application/json'
        
    response = req.request(method, api+endpoint, data=payload, headers=headers)   
    status_code = response.status_code
    
    if method in ["GET","POST"]:
        response = response.json()

    return response, status_code

### 4. Check if Domain exists

In [ ]:
def get_domain_id_if_exists(domain_name, ob_domains):
    for d in ob_domains:
        if d["domain"] == domain_name:
            return d["id"]
    return False

### 5. Check if Audit exists

In [ ]:
def get_audit_id_if_exists(domain_name, ob_audits):
    for a in ob_audits:
        if a["name"] == domain_name:
            return a["id"]
    return False 

### 6. Create Domain

In [ ]:
def create_domain(domain_name):
    domain_response , _ = get_data(
        "domains",
        "POST",
        json.dumps({
          'name': domain_name,
          'domain': domain_name
        })
    )
    domain_id = domain_response["id"]
    
    return domain_id

### 7. Create Audit

In [ ]:
def create_audit(domain_id, domain_name, template_audit_id):
    template_audit , _ = get_data(f"web-audits/{template_audit_id}")
    template_audit["name"] = domain_name
    template_audit["startingUrls"] = [f'https://{domain_name}']
    template_audit["filters"]["include"][0]["value"] = f'^https?://(www\.)?{domain_name}'
    template_audit["domainId"] = domain_id


    audit_response , _ = get_data("web-audits","POST",json.dumps(template_audit))    
    audit_id = audit_response["id"]

    ob_template_audit_actions , _ = get_data(f"web-audits/{template_audit_id}/actions")
    
    action_response , action_status = get_data(
        f"web-audits/{audit_id}/actions","PUT",
        json.dumps({
            "actions": ob_template_audit_actions
        })
    )
    
    return audit_id

### 8. Get all Domains in ObservePoint

In [ ]:
def get_domains():
    return get_data("domains")[0]

### 9. Get all Audits in ObservePoint

In [ ]:
def get_audits():
    ob_audits =  get_data("web-audits")[0]
    
    return ob_audits

### 10. Create Audits

In [ ]:
def create_audits(list_of_domains, template_audit_id):
    ob_domains = get_domains()
    ob_audits = get_audits()

    for domain_name in list_of_domains:
        domain_id = get_domain_id_if_exists(domain_name, ob_domains)
        
        if not domain_id:
            domain_id = create_domain(domain_name)
            print(f"domain {domain_name} created and assigned id {domain_id} ", sep="")
    
        audit_id = get_audit_id_if_exists(domain_name, ob_audits)
        if not audit_id:
            audit_id = create_audit(domain_id, domain_name, template_audit_id)
            print(f"audit for {domain_name} created and assigned id {audit_id}")
    
    
doamin_list = ['domain1', 'domain2']
create_audits(doamin_list, template_audit_id=template_audit_id)